# Required Installations

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


# Colors

In [ ]:
colors = ['red', 'blue', 'yellow', 'pink', 'white', 'black', 'green', 'orange', 'purple', 'brown']

In [ ]:
# The colors we consider for this dataset and the number of common sense
# examples that mention them.
def color_freq_dict():
  color_freq = {
      'red': 0,
      'blue': 0,
      'yellow': 0,
      'pink': 0,
      'white': 0,
      'black': 0,
      'green': 0,
      'orange': 0,
      'purple': 0,
      'brown': 0
  }
  return color_freq

In [ ]:
# Prints the color and how often samples containing it are seen.
def print_freq(color_freq, colors):
  for color in colors:
    print(color+': '+str(color_freq[color]))

# ConceptNet API

We know that in this API, the `surfaceText` param is the natural language data on which the relationship is based. Further, we know that sentences must either begin with `[[color]]` or end with `[[color]]`. So, we request the API for sentences starting and ending with each of the colors under consideration.



In [ ]:
import requests

In [ ]:
def find_color(query_str, color_freq, colors):
  examples = [] # list of all examples included in our dataset

  for color in colors:
    current_query = query_str.format(color)

    current_obj = requests.get(current_query).json()

    all_current_exs = current_obj['edges']

    for ex in all_current_exs:
      surface_text = ex['surfaceText']

      if surface_text is not None:

        # Sentences appear in the form "[[red]] is a [[color]]"
        surface_text = surface_text.replace('[', '')
        surface_text = surface_text.replace(']', '')

        # Converts the sentence into lower case
        surface_text = surface_text.lower()

        examples.append(surface_text)

        color_freq[color] += 1

  return examples

In [ ]:
# sample URL query for interfacing with the API
# sentences that start with the color
query_str = 'http://api.conceptnet.io/query?start=/c/en/{}&limit=1000'

start_color_freq = color_freq_dict()

start_examples = find_color(query_str, start_color_freq, colors)

In [ ]:
print_freq(start_color_freq, colors)

red: 80
blue: 202
yellow: 28
pink: 12
white: 41
black: 69
green: 108
orange: 60
purple: 18
brown: 10


In [ ]:
# sample URL query for interfacing with the API
# sentences that end with the color
query_str = 'http://api.conceptnet.io/query?end=/c/en/{}&limit=1000'

end_color_freq = color_freq_dict()

end_examples = find_color(query_str, end_color_freq, colors)

In [ ]:
print_freq(end_color_freq, colors)

red: 109
blue: 102
yellow: 105
pink: 99
white: 160
black: 133
green: 195
orange: 144
purple: 67
brown: 129


In [ ]:
with open('conceptnet.txt', 'a') as f:
  for ex in start_examples:
    f.write(ex + '\n')

  for ex in end_examples:
    f.write(ex + '\n')

# OMCS Common Sense Dataset

In [ ]:
from datasets import load_dataset

In [ ]:
from tqdm import tqdm

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import string

In [ ]:
dataset = load_dataset("dutta18/omcs_dataset_of_commonsense_facts")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1578238 [00:00<?, ? examples/s]

In [ ]:
omcs_examples = []

In [ ]:
color_freq = color_freq_dict()

In [ ]:
# Iterates through all facts in the OMCS dataset and checks if it contains
# a color in our list.
# If it does, this sentence is appended to our list of examples.
for sample in tqdm(dataset["train"]):

  # Extracts fact from the sample.
  fact = sample['fact']

  # Iterates through all colors.
  for color in colors:

    fact_tokens = word_tokenize(fact)
    # Checks if any color is mentioned in the fact.
    if color in fact_tokens:

      # Converts sentence to lowercase and removes all punctuation.
      fact = fact.lower()
      fact = fact.translate(str.maketrans('', '', string.punctuation))

      omcs_examples.append(fact)

      # Increases the number of examples for a given color by 1.
      color_freq[color] += 1
      break

100%|██████████| 1578238/1578238 [30:54<00:00, 850.93it/s] 


In [ ]:
with open('omcs_dataset.txt', 'a') as f:
  for fact in omcs_examples:
    f.write(fact+'\n')

In [ ]:
print_freq(color_freq, colors)

red: 2968
blue: 1880
yellow: 1382
pink: 283
white: 3040
black: 1951
green: 1992
orange: 890
purple: 320
brown: 859
